In [178]:
import argparse
import io
import json
import os
import sys
from typing import Final, List

import pandas as pd
import pendulum
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from requests_kerberos import OPTIONAL, HTTPKerberosAuth
urllib3.disable_warnings()

import datetime
from datetime import datetime, timedelta


In [179]:
today = datetime.now()

count = 0

whid = ['VIL2', 'VIL3', 'VMI1', 'VFL4', 'VIL1']

In [180]:
def intro():
    global count
    while count < 1:
        print("""
                Welcome to the Volume Retriever v2.0
                Brought to you by: Gerber Albizures (galbizur@amazon.com)
                New to v2.0, recursion introduced for VIL2, VIL3, SMI1, VFL4, VIL1
                Multithreading reducing overall times from over 10 min to just under 2!
                Excel color map created for visualization purposes to emulate excel processing
                Visualizations will be saved to produce email.
        """)
    #global count
        count = count + 1
    
intro()


                Welcome to the Volume Retriever v2.0
                Brought to you by: Gerber Albizures (galbizur@amazon.com)
                New to v2.0, recursion introduced for VIL2, VIL3, SMI1, VFL4, VIL1
                Multithreading reducing overall times from over 10 min to just under 2!
                Excel color map created for visualization purposes to emulate excel processing
                Visualizations will be saved to produce email.
        


In [181]:
def dow(date):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    dayNumber=date.weekday()
    print(dayNumber)
    return dayNumber
    
def getSaturday():
    today = datetime.now()
    date = today.weekday()
    if date == 0:
        d = today - timedelta(days=2)
        return d
    elif date == 1:
        d = today - timedelta(days=3)
        #print(type(d))
        return d
    elif date == 2:
        d = today - timedelta(days=4)
        return d
    elif date == 3:
        d = today - timedelta(days=5)
        return d
    elif date == 4:
        d = today - timedelta(days=6)
        return d
    elif date == 5:
        d = today - timedelta(days=7)
        return d
    else:
        d = today - timedelta(days=1)
        return d
        

lastSaturday = (getSaturday())
lastSunday = getSaturday() - timedelta(days=6)

strLastSaturday = lastSaturday.strftime('%Y-%m-%d')


strLastSunday = lastSunday.strftime('%Y-%m-%d')


In [182]:
def SundayVolumeAM():
    if count == 1:
        weekDay = "Sunday"
        
        print(str(datetime.now())+": Starting Volume pulls from last", weekDay, "day shift.")
        date1 = strLastSunday
        year1, month1, day1 = map(str, strLastSunday.split('-'))
        time1 = "06:30"
        hour1, minute1 = map(str, time1.split(':'))

        date2 = date1
        year2, month2, day2 = map(str, date2.split('-'))
        time2 = "20:00"
        hour2, minute2 = map(str, time2.split(':'))
        
        dateTotal = str(date1)+ " "+str(time1)+"-" +str(time2)
        my_dict = {"DS":[], dateTotal:[]}
   
        for x in whid:
            fclm_url = f'http://fclm-portal.amazon.com/ppa/inspect/node/csv?nodeType=DS&warehouseId={x}&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F06%2F06&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday={year1}%2F{month1}%2F{day1}&startHourIntraday={hour1}&startMinuteIntraday={minute1}&endDateIntraday={year2}%2F{month2}%2F{day2}&endHourIntraday={hour2}&endMinuteIntraday={minute2}'
            req = requests.Session()
            resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
            data = io.StringIO(resp.text)
            df = pd.read_csv(data)
    #print(df.info)
            lfilter = df['Labor Process Name']=="Container Building" 
            df.where(lfilter, inplace=True)#1
            filter = df['Main Processes'] == 'In-Station Core'
            df.where(filter, inplace=True)
            DS_sum = df["Actual Units"].sum()
            
            if x == whid[-1] and today.isocalendar()[1] < 31:               

                if DS_sum == 0:
                    print(str(datetime.now())+":", weekDay, "day shift pull request for", x, "dropped, retry in place.")
                    req = requests.Session()
                    resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                    data = io.StringIO(resp.text)
                    df = pd.read_csv(data)
                #print(df.info)
                    lfilter = df['Labor Process Name']=="Container Building" 
                    df.where(lfilter, inplace=True)#2
                    filter = df['Main Processes'] == 'In-Station Core'
                    df.where(filter, inplace=True)
                    DS_sum = df["Actual Units"].sum()
                print(str(datetime.now())+":", weekDay, "pull for", x, "completed.")
                
            else:
                while DS_sum == 0:
                    print(str(datetime.now())+":", weekDay, "day shift pull request for", x, "dropped, retry in place.")
                    req = requests.Session()
                    resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                    data = io.StringIO(resp.text)
                    df = pd.read_csv(data)
                    #print(df.info)
                    lfilter = df['Labor Process Name']=="Container Building" 
                    df.where(lfilter, inplace=True)#2
                    filter = df['Main Processes'] == 'In-Station Core'
                    df.where(filter, inplace=True)
                    DS_sum = df["Actual Units"].sum()
                print(str(datetime.now())+":", weekDay, "pull for", x, "completed.")
                    
      
            my_dict["DS"].append(x)
            my_dict[dateTotal].append(DS_sum)
        
        Volume1 = pd.DataFrame.from_dict(my_dict, orient='index', columns = ['VIL2', 'VIL3', 'VMI1', 'VFL4', 'VIL1'])
        Volume1.drop(labels = 'DS', axis = 0, inplace=True)

        Volume1 = Volume1.reset_index()
        Volume1.to_csv("Volume1.csv")

        return Volume1
v1 = SundayVolumeAM()

v1
        

2021-07-29 01:31:05.324189: Starting Volume pulls from last Sunday day shift.
2021-07-29 01:31:09.462385: Sunday pull for VIL2 completed.
2021-07-29 01:31:31.514922: Sunday pull for VIL3 completed.
2021-07-29 01:31:44.725086: Sunday pull for VMI1 completed.
2021-07-29 01:31:57.979761: Sunday pull for VFL4 completed.
2021-07-29 01:32:05.733981: Sunday pull request for VIL1 dropped, retry in place.
2021-07-29 01:32:18.327229: Sunday pull for VIL1 completed.


,index,VIL2,VIL3,VMI1,VFL4,VIL1
0,2021-07-18 06:30-20:00,11546.0,7270.0,10808.0,10033.0,0


In [183]:
v1

,index,VIL2,VIL3,VMI1,VFL4,VIL1
0,2021-07-18 06:30-20:00,11546.0,7270.0,10808.0,10033.0,0
